# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace/udacity-nanodegree-data-eng/project2
['/home/workspace/udacity-nanodegree-data-eng/project2/event_data/2018-11-27-events.csv', '/home/workspace/udacity-nanodegree-data-eng/project2/event_data/2018-11-04-events.csv', '/home/workspace/udacity-nanodegree-data-eng/project2/event_data/2018-11-07-events.csv', '/home/workspace/udacity-nanodegree-data-eng/project2/event_data/2018-11-09-events.csv', '/home/workspace/udacity-nanodegree-data-eng/project2/event_data/2018-11-19-events.csv', '/home/workspace/udacity-nanodegree-data-eng/project2/event_data/2018-11-05-events.csv', '/home/workspace/udacity-nanodegree-data-eng/project2/event_data/2018-11-22-events.csv', '/home/workspace/udacity-nanodegree-data-eng/project2/event_data/2018-11-16-events.csv', '/home/workspace/udacity-nanodegree-data-eng/project2/event_data/2018-11-26-events.csv', '/home/workspace/udacity-nanodegree-data-eng/project2/event_data/2018-11-24-events.csv', '/home/workspace/udacity-nanodegree-data-eng/project2/ev

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88'], ['Jimi Hendrix', 'Logged In', 'Mohammad', 'M', '1', 'Rodriguez', '239.82975', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Woodstock Inprovisation', '200', '1.54328E+12', '88'], ['Building 429', 'Logged In', 'Mohammad', 'M', '2', 'Rodriguez', '300.61669', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Majesty (LP Version)', '200', '1.54328E+12', '88'], ["The B-52's", 'Logged In', 'Gianna', 'F', '0', 'Jones', '321.54077', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54087E+12', '107', 'Love Shack', '200', '1.54328E+12', '38'], ['Die Mooskirc

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# TO-DO: Create a Keyspace 
session.execute("""
CREATE KEYSPACE IF NOT EXISTS udacity 
WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [9]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS music_events "
query = query + """(artist text, song text, length float, sessionId int, itemInSession int, 
PRIMARY KEY (sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)


In [32]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_events (artist, song, length, sessionId, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner
Mohammad
M
0
Rodriguez
277.15873
paid
Sacramento--Roseville--Arden-Arcade, CA
<class 'str'>
Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)
88
Jimi Hendrix
Mohammad
M
1
Rodriguez
239.82975
paid
Sacramento--Roseville--Arden-Arcade, CA
<class 'str'>
Woodstock Inprovisation
88
Building 429
Mohammad
M
2
Rodriguez
300.61669
paid
Sacramento--Roseville--Arden-Arcade, CA
<class 'str'>
Majesty (LP Version)
88
The B-52's
Gianna
F
0
Jones
321.54077
free
New York-Newark-Jersey City, NY-NJ-PA
<class 'str'>
Love Shack
38
Die Mooskirchner
Gianna
F
1
Jones
169.29914
free
New York-Newark-Jersey City, NY-NJ-PA
<class 'str'>
Frisch und g'sund
38
Carrie Underwood
Gianna
F
2
Jones
195.70893
free
New York-Newark-Jersey City, NY-NJ-PA
<class 'str'>
Look At Me
38
No Te Va Gustar
Kaylee
F
0
Summers
246.43873
free
Phoenix-Mesa-Scottsdale, AZ
<class 'str'>
Ya EntendÃÂ­
8
The Velvet Underground / Nico
Kaylee
F
1
Summers
36

#### Do a SELECT to verify that the data have been inserted into each table

In [35]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
              
query = "select artist, song, length  from music_events WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length,)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [49]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


query = "CREATE TABLE IF NOT EXISTS user_music_events "
query = query + """(artist text, song text, firstname text, lastname text, userId int, sessionId int, itemInSession int, 
PRIMARY KEY ((userid, sessionId), itemInSession))"""

session.execute(query)     

In [52]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_music_events (artist, song, firstname, lastname, userId, sessionId, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [53]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
              
query = "select artist, song, firstname, lastname from user_music_events WHERE  userId = 10 AND sessionId = 182 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname,row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [47]:
query = "drop table user_music_events"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()